# 1. EDA (Exploratory Data Analysis)

## Bank Marketing Dataset - Vadeli Mevduat Tahmini

Bu notebook, Portekiz bankasinin telefon pazarlama kampanyasi verisinin detayli incelemesini icerir.

**Amac:** Musterilerin vadeli mevduat (term deposit) acip acmayacagini tahmin etmek.

**Kaynak:** https://www.kaggle.com/datasets/janiobachmann/bank-marketing-dataset

## Icerik
1. Veri Yukleme ve Ilk Bakis
2. Target Analizi
3. Eksik Deger Analizi
4. Numerik Degisken Analizi
5. Kategorik Degisken Analizi
6. Korelasyon Analizi
7. Feature-Target Iliskisi
8. Bulgular ve Sonuc

In [ ]:
# Kutuphaneler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Gorsellestirme ayarlari
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12
sns.set_palette('husl')

# Pandas ayarlari
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

print('Kutuphaneler yuklendi!')

---
## 1. Veri Yukleme ve Ilk Bakis

In [ ]:
# Veri yolu
DATA_PATH = Path('../data/raw/')

# Veriyi yukle
df = pd.read_csv(DATA_PATH / 'bank.csv')
print(f'Veri yuklendi: {df.shape[0]:,} satir, {df.shape[1]} sutun')

In [ ]:
# Ilk 5 satir
df.head()

In [ ]:
# Son 5 satir
df.tail()

In [ ]:
# Veri tipleri ve bellek kullanimi
df.info()

In [ ]:
# Sutun isimleri
print('Sutunlar:')
for i, col in enumerate(df.columns, 1):
    print(f'{i:2}. {col}')

### Degisken Aciklamalari

| Degisken | Tip | Aciklama |
|----------|-----|----------|
| age | Numerik | Musteri yasi |
| job | Kategorik | Meslek turu |
| marital | Kategorik | Medeni durum (married, single, divorced) |
| education | Kategorik | Egitim seviyesi |
| default | Kategorik | Kredi temerrut durumu |
| balance | Numerik | Yillik ortalama bakiye (Euro) |
| housing | Kategorik | Konut kredisi var mi? |
| loan | Kategorik | Bireysel kredi var mi? |
| contact | Kategorik | Iletisim turu |
| day | Numerik | Son iletisim gunu (ayin kacinci gunu) |
| month | Kategorik | Son iletisim ayi |
| duration | Numerik | Son gorusme suresi (saniye) - **DIKKAT: Production'da bilinmez!** |
| campaign | Numerik | Bu kampanyada yapilan arama sayisi |
| pdays | Numerik | Onceki kampanyadan bu yana gecen gun (-1: hic aranmamis) |
| previous | Numerik | Onceki kampanyalarda yapilan arama sayisi |
| poutcome | Kategorik | Onceki kampanya sonucu |
| **deposit** | **Target** | **Vadeli mevduat acti mi? (yes/no)** |

---
## 2. Target Analizi

In [ ]:
# Target degisken
TARGET = 'deposit'

# Target dagilimi
print('Target Dagilimi:')
print(df[TARGET].value_counts())
print('\nTarget Oranlari:')
print(df[TARGET].value_counts(normalize=True).round(4) * 100)

In [ ]:
# Target dagilimi gorsellestirme
fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart
colors = ['#e74c3c', '#2ecc71']
df[TARGET].value_counts().plot(kind='bar', ax=ax[0], color=colors, edgecolor='black')
ax[0].set_title('Vadeli Mevduat Dagilimi (Sayi)', fontsize=14, fontweight='bold')
ax[0].set_xlabel('Deposit')
ax[0].set_ylabel('Musteri Sayisi')
ax[0].set_xticklabels(['Hayir (no)', 'Evet (yes)'], rotation=0)

# Degerler ekle
for i, v in enumerate(df[TARGET].value_counts().values):
    ax[0].text(i, v + 100, f'{v:,}', ha='center', fontweight='bold')

# Pie chart
df[TARGET].value_counts().plot(kind='pie', ax=ax[1], autopct='%1.1f%%', 
                                colors=colors, explode=(0.02, 0.02),
                                shadow=True, startangle=90)
ax[1].set_title('Vadeli Mevduat Dagilimi (Oran)', fontsize=14, fontweight='bold')
ax[1].set_ylabel('')

plt.tight_layout()
plt.savefig('../docs/target_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n>>> Grafik kaydedildi: docs/target_distribution.png')

### Target Analizi Bulgulari

- **Veri seti dengeli gorunuyor** (yaklasik 47% yes, 53% no)
- Dengesiz veri problemleri icin ozel teknikler (SMOTE, class weights vb.) gerekli olmayabilir
- Ancak yine de stratified sampling kullanmak iyi bir pratiktir

---
## 3. Eksik Deger Analizi

In [ ]:
# Eksik deger sayilari
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100

missing_df = pd.DataFrame({
    'Eksik Sayi': missing,
    'Eksik Oran (%)': missing_pct.round(2)
}).sort_values('Eksik Oran (%)', ascending=False)

print('Eksik Deger Analizi:')
print(f'Toplam eksik deger: {missing.sum()}')
print(f'\nSutun bazinda eksik degerler:')
missing_df

In [ ]:
# "unknown" degerlerini kontrol et (eksik deger gibi davranabilir)
print('"unknown" Degerleri:')
for col in df.select_dtypes(include='object').columns:
    unknown_count = (df[col] == 'unknown').sum()
    if unknown_count > 0:
        print(f'{col}: {unknown_count:,} ({unknown_count/len(df)*100:.2f}%)')

### Eksik Deger Bulgulari

- **Teknik olarak eksik deger YOK** (null/NaN)
- Ancak bazi kategorik degiskenlerde **"unknown"** degeri var:
  - `contact`: Iletisim turu bilinmiyor
  - `poutcome`: Onceki kampanya sonucu bilinmiyor
  - `education`: Egitim seviyesi bilinmiyor
- **pdays = -1**: Musteri daha once hic aranmamis (ozel bir deger)

**Strateji:**
- "unknown" degerlerini ayri bir kategori olarak tutabiliriz
- veya mode ile doldurma dusunulebilir (EDA sonrasi karar verilecek)

---
## 4. Numerik Degisken Analizi

In [ ]:
# Numerik sutunlar
num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
print(f'Numerik sutun sayisi: {len(num_cols)}')
print(f'Sutunlar: {num_cols}')

In [ ]:
# Temel istatistikler
df[num_cols].describe().T.round(2)

In [ ]:
# Dagilim grafikleri
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.flatten()

for i, col in enumerate(num_cols):
    ax = axes[i]
    df[col].hist(bins=50, ax=ax, color='steelblue', edgecolor='black', alpha=0.7)
    ax.set_title(f'{col}', fontsize=12, fontweight='bold')
    ax.set_xlabel('')
    ax.axvline(df[col].mean(), color='red', linestyle='--', label=f'Mean: {df[col].mean():.1f}')
    ax.axvline(df[col].median(), color='green', linestyle='-', label=f'Median: {df[col].median():.1f}')
    ax.legend(fontsize=8)

# Bos subplot'lari kaldir
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.suptitle('Numerik Degiskenlerin Dagilimi', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('../docs/numeric_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Boxplot ile outlier analizi
fig, axes = plt.subplots(3, 3, figsize=(15, 10))
axes = axes.flatten()

for i, col in enumerate(num_cols):
    ax = axes[i]
    df.boxplot(column=col, ax=ax)
    ax.set_title(f'{col}', fontsize=12, fontweight='bold')

# Bos subplot'lari kaldir
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.suptitle('Numerik Degiskenlerin Boxplot Analizi (Outlier Tespiti)', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Outlier istatistikleri
print('Outlier Analizi (IQR Yontemi):')
print('-' * 50)

for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    
    outliers = df[(df[col] < lower) | (df[col] > upper)][col]
    if len(outliers) > 0:
        print(f'{col}: {len(outliers):,} outlier ({len(outliers)/len(df)*100:.2f}%)')

### Numerik Degisken Bulgulari

1. **age (Yas):**
   - Dagilim normal'e yakin
   - Ortalama: ~41, Min: 18, Max: 95

2. **balance (Bakiye):**
   - Saga carpik (right-skewed) dagilim
   - Negatif bakiyeler var (borclu musteriler)
   - Outlier'lar mevcut

3. **duration (Gorusme Suresi):**
   - **ONEMLI:** Bu feature production'da bilinmez!
   - Saga carpik dagilim
   - Uzun gorusmeler genellikle basariyla sonuclaniyor

4. **campaign (Arama Sayisi):**
   - Cogu musteri 1-3 kez aranmis
   - Bazi musteriler cok fazla aranmis (outlier)

5. **pdays:**
   - -1 degeri: Daha once hic aranmamis
   - Bu ozel bir flag olarak kullanilabilir

6. **previous:**
   - Cogu musteri daha once hic aranmamis (0)

---
## 5. Kategorik Degisken Analizi

In [ ]:
# Kategorik sutunlar
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f'Kategorik sutun sayisi: {len(cat_cols)}')
print(f'Sutunlar: {cat_cols}')

In [ ]:
# Kardinalite (benzersiz deger sayisi)
print('Kategorik Degisken Kardinaliteleri:')
print('-' * 40)
for col in cat_cols:
    print(f'{col}: {df[col].nunique()} benzersiz deger')
    print(f'   Degerler: {df[col].unique()}')
    print()

In [ ]:
# Kategorik degisken dagilimi
fig, axes = plt.subplots(3, 3, figsize=(16, 14))
axes = axes.flatten()

for i, col in enumerate(cat_cols):
    ax = axes[i]
    order = df[col].value_counts().index
    sns.countplot(data=df, x=col, ax=ax, order=order, palette='viridis')
    ax.set_title(f'{col}', fontsize=12, fontweight='bold')
    ax.set_xlabel('')
    ax.tick_params(axis='x', rotation=45)
    
    # Deger etiketleri
    for p in ax.patches:
        ax.annotate(f'{int(p.get_height()):,}', 
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='bottom', fontsize=8)

# Bos subplot'lari kaldir
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.suptitle('Kategorik Degiskenlerin Dagilimi', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('../docs/categorical_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

### Kategorik Degisken Bulgulari

1. **job (Meslek):**
   - En yaygin: blue-collar, management, technician
   - 12 farkli meslek kategorisi

2. **marital (Medeni Durum):**
   - Cogunluk evli (married)
   - 3 kategori: married, single, divorced

3. **education (Egitim):**
   - En yaygin: secondary
   - "unknown" degerleri var

4. **default (Temerrut):**
   - Buyuk cogunluk "no"
   - Dengesiz sinif

5. **housing (Konut Kredisi):**
   - Yaklasik yarisinin konut kredisi var

6. **loan (Bireysel Kredi):**
   - Cogunlugun bireysel kredisi yok

7. **contact (Iletisim Turu):**
   - cellular en yaygin
   - "unknown" orani yuksek

8. **month (Ay):**
   - Mayis ayi en yogun
   - Mevsimsellik olabilir

9. **poutcome (Onceki Kampanya Sonucu):**
   - "unknown" orani cok yuksek
   - Cogu musteri ilk kez araniyor

---
## 6. Korelasyon Analizi

In [ ]:
# Target'i numerik yap
df['deposit_numeric'] = (df['deposit'] == 'yes').astype(int)

# Korelasyon matrisi
corr_cols = num_cols + ['deposit_numeric']
corr_matrix = df[corr_cols].corr()

# Heatmap
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', 
            cmap='RdBu_r', center=0, square=True,
            linewidths=0.5, cbar_kws={'shrink': 0.8})
plt.title('Korelasyon Matrisi', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('../docs/correlation_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Target ile korelasyon
target_corr = corr_matrix['deposit_numeric'].drop('deposit_numeric').sort_values(ascending=False)

print('Target (deposit) ile Korelasyonlar:')
print('=' * 40)
for feature, corr in target_corr.items():
    direction = '+' if corr > 0 else ''
    print(f'{feature:15} : {direction}{corr:.4f}')

In [ ]:
# Target korelasyon bar chart
plt.figure(figsize=(10, 6))
colors = ['green' if x > 0 else 'red' for x in target_corr.values]
target_corr.plot(kind='barh', color=colors, edgecolor='black')
plt.title('Feature-Target Korelasyonu', fontsize=14, fontweight='bold')
plt.xlabel('Korelasyon Katsayisi')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.tight_layout()
plt.show()

### Korelasyon Bulgulari

**Target ile Pozitif Korelasyon:**
- `duration`: En guclu korelasyon (ama production'da kullanamayiz!)
- `previous`: Onceki kampanyalarda aranma sayisi
- `pdays`: Onceki kampanyadan bu yana gecen gun

**Target ile Negatif Korelasyon:**
- `campaign`: Cok arama = dusuk donusum
- `day`: Ayin gunu (zayif)

**Feature'lar Arasi Yuksek Korelasyon:**
- `pdays` ve `previous`: Beklenen (onceki kampanya bilgileri)

**Onemli Not:**
`duration` feature'i en yuksek korelasyona sahip ama **production'da kullanilamaz!**

---
## 7. Feature-Target Iliskisi (Detayli)

In [ ]:
# Kategorik degiskenler vs Target
fig, axes = plt.subplots(3, 3, figsize=(16, 14))
axes = axes.flatten()

for i, col in enumerate(cat_cols[:-1]):  # deposit haric
    ax = axes[i]
    
    # Crosstab orani
    ct = pd.crosstab(df[col], df['deposit'], normalize='index') * 100
    ct['yes'].sort_values(ascending=True).plot(kind='barh', ax=ax, color='green', edgecolor='black')
    
    ax.set_title(f'{col} vs Deposit Rate', fontsize=12, fontweight='bold')
    ax.set_xlabel('Deposit Orani (%)')
    ax.axvline(x=df['deposit_numeric'].mean()*100, color='red', linestyle='--', label='Ortalama')
    ax.legend()

# Bos subplot'lari kaldir
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.suptitle('Kategorik Degiskenler vs Deposit Orani', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('../docs/categorical_vs_target.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Numerik degiskenler vs Target (boxplot)
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.flatten()

for i, col in enumerate(num_cols):
    ax = axes[i]
    df.boxplot(column=col, by='deposit', ax=ax)
    ax.set_title(f'{col}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Deposit')

# Bos subplot'lari kaldir
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.suptitle('Numerik Degiskenler vs Deposit', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('../docs/numeric_vs_target.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Yas gruplari analizi
df['age_group'] = pd.cut(df['age'], bins=[0, 30, 40, 50, 60, 100], 
                         labels=['18-30', '31-40', '41-50', '51-60', '60+'])

age_deposit = pd.crosstab(df['age_group'], df['deposit'], normalize='index') * 100

plt.figure(figsize=(10, 6))
age_deposit.plot(kind='bar', stacked=True, color=['#e74c3c', '#2ecc71'], edgecolor='black')
plt.title('Yas Gruplarina Gore Deposit Orani', fontsize=14, fontweight='bold')
plt.xlabel('Yas Grubu')
plt.ylabel('Oran (%)')
plt.legend(title='Deposit', loc='upper right')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

print('Yas Gruplarina Gore Deposit Orani:')
print(age_deposit['yes'].round(2))

### Feature-Target Iliskisi Bulgulari

**Yuksek Donusum Oranlari:**
- `poutcome = success`: Onceki kampanyada basarili olanlar
- `contact = cellular`: Cep telefonundan ulasilanlar
- `job = student, retired`: Ogrenciler ve emekliler
- `month = mar, sep, oct, dec`: Bazi aylar daha iyi

**Dusuk Donusum Oranlari:**
- `poutcome = failure`: Onceki kampanyada basarisiz olanlar
- `job = blue-collar`: Mavi yakalilar
- `default = yes`: Temerrut gecmisi olanlar

**Yas Analizi:**
- Genc (18-30) ve yasli (60+) musteriler daha yuksek donusum
- Orta yaslilar (31-50) daha dusuk donusum

---
## 8. Bulgular ve Sonuc

In [ ]:
# Gecici sutunlari kaldir
df.drop(['deposit_numeric', 'age_group'], axis=1, inplace=True, errors='ignore')

# Ozet istatistikler
print('='*60)
print('VERI SETI OZETI')
print('='*60)
print(f'Toplam satir: {len(df):,}')
print(f'Toplam sutun: {len(df.columns)}')
print(f'Numerik sutun: {len(num_cols)}')
print(f'Kategorik sutun: {len(cat_cols)}')
print(f'Eksik deger: {df.isnull().sum().sum()}')
print(f'\nTarget Dagilimi:')
print(f'  - Yes (Deposit): {(df["deposit"]=="yes").sum():,} ({(df["deposit"]=="yes").mean()*100:.1f}%)')
print(f'  - No (No Deposit): {(df["deposit"]=="no").sum():,} ({(df["deposit"]=="no").mean()*100:.1f}%)')
print('='*60)

## Onemli Bulgular Ozeti

### 1. Veri Kalitesi
- Eksik deger YOK (teknik olarak)
- "unknown" degerleri bazi sutunlarda mevcut
- Outlier'lar balance ve duration'da belirgin

### 2. Target Dagilimi
- Yaklasik dengeli (~47% yes, ~53% no)
- Ozel dengeleme teknikleri zorunlu degil

### 3. Onemli Feature'lar
- **duration**: En guclu tahmin edici AMA production'da KULLANILAMAZ!
- **poutcome**: Onceki kampanya basarisi cok onemli
- **contact**: Iletisim turu etkili
- **month**: Mevsimsellik mevcut

### 4. Feature Engineering Onerileri
- Yas gruplari olusturma
- Bakiye kategorileri
- pdays icin "hic aranmadi" flag'i
- Ay bazli mevsimsellik
- Kampanya yogunlugu metrikleri

### 5. Dikkat Edilecekler
- `duration` feature'i gercekci bir production modeli icin **cikarilmali**
- "unknown" degerleri ozel olarak ele alinmali
- Stratified sampling kullanilmali

---

## Sonraki Adimlar

1. **Baseline Model** (02_baseline.ipynb)
   - Basit preprocessing
   - LightGBM ile baseline skor
   
2. **Feature Engineering** (03_feature_engineering.ipynb)
   - Yukaridaki onerilerin uygulanmasi
   - Feature selection

In [ ]:
print('EDA tamamlandi!')
print('Sonraki adim: notebooks/02_baseline.ipynb')